In [ ]:
from pystac_client import Client
from odc.stac import load

import odc.geo.xr

In [ ]:
# STAC Catalog URL
catalog = "https://earth-search.aws.element84.com/v1"

# Create a STAC Client
client = Client.open(catalog)

In [ ]:
# Find a location you're interested in on Google Maps and copy the coordinates
# by right-clicking on the map and clicking the coordinates

# These coords are in the order Y then X, or Latitude then Longitude
# Area in Vietnam
coords = 9.983038785699682, 106.73711560238493
buffer = 0.2
bbox = (coords[1] - buffer, coords[0] - buffer, coords[1] + buffer, coords[0] + buffer)

datetime = "2024-07"

In [ ]:
items = client.search(
    collections=["sentinel-2-c1-l2a"],
    bbox=bbox,
    datetime=datetime,
    query={"eo:cloud_cover": {"lt": 80}},  # Not using this filter results in 27 items
).item_collection()

print(f"Found {len(items)} STAC items")

In [ ]:
data = load(
    items,
    bands=["red", "green", "blue"],
    bbox=bbox,
    groupby="solar_day",
    chunks={"x": 2048, "y": 2048},
)

data = data.load()

In [ ]:
data.to_array().plot.imshow(col="time", col_wrap=2, size=6, vmin=1000, vmax=4000)

In [ ]:
best = data.sel(time="2024-07-30").squeeze()

best.odc.explore(vmin=1000, vmax=4000)

In [ ]:
out = best.odc.to_rgba(vmin=1000, vmax=4000)

odc.geo.xr.write_cog(out, "output.tif", overwrite=True)